# Sentiment Analysis - Labelled Financial News Data Using Transformer Models


In [25]:
# Importing the required libraries
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import Trainer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
import evaluate
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

### Loading the data

In [26]:
data = pd.read_csv('data/Fin_cleaned.csv', encoding='utf-8')

# duplicate the data
news = data.copy()

news

,Date_published,Headline,Synopsis,Full_text,Final Status
0,2022-06-21,"Banks holding on to subsidy share, say payment...",The companies have written to the National Pay...,ReutersPayments companies and banks are at log...,Negative
1,2022-04-19,Digitally ready Bank of Baroda aims to click o...,"At present, 50% of the bank's retail loans are...",AgenciesThe bank presently has 20 million acti...,Positive
2,2022-05-27,Karnataka attracted investment commitment of R...,Karnataka is at the forefront in attracting in...,PTIKarnataka Chief Minister Basavaraj Bommai.K...,Positive
3,2022-04-06,Splitting of provident fund accounts may be de...,The EPFO is likely to split accounts only at t...,Getty ImagesThe budget for FY22 had imposed in...,Negative
4,2022-06-14,Irdai weighs proposal to privatise Insurance I...,"Set up in 2009 as an advisory body, IIB collec...",AgenciesThere is a view in the insurance indus...,Positive
...,...,...,...,...,...
395,2022-06-10,"Banks take a cue from RBI, hike lending rates",These banks raised their respective external b...,"PTIICICI Bank, Bank of Baroda, Punjab National...",Negative
396,2022-06-29,Sebi issues Rs 27 lakh recovery notice to indi...,"In the event of non-payment, it will recover t...",ReutersThe logo of the Securities and Exchange...,Negative
397,2022-06-06,Apollo Hospital shares drop 0.68% as Sensex ...,"A total of 10,105 shares changed hands on the ...",Getty ImagesShrikant Chouhan of Kotak Securiti...,Negative
398,2022-05-16,SBI at Rs 710? What makes analysts see up to 5...,Calling the stock 'attractively valued' analys...,AgenciesThe PSU bank reported a 41.27 per cent...,Positive


### Data Preprocessing

In [27]:
# Fill the missing value with an empty string
news['Synopsis'] = news['Synopsis'].fillna('')

# Rename columns
news = news.rename(
    columns={'Date_published': 'date', 'Headline': 'headline', 'Synopsis': 'synopsis', 'Full_text': 'text',
             'Final Status': 'label'})

# Remove the spaces from the labels
news['label'] = news['label'].str.strip()

# Combine the headline, synopsis, and text columns
news['full_text'] = news['headline'] + ' ' + news['synopsis'] + ' ' + news['text']

# Convert labels to binary
news['label'] = news['label'].apply(lambda x: 1 if x == 'Positive' else 0)

news

,date,headline,synopsis,text,label,full_text
0,2022-06-21,"Banks holding on to subsidy share, say payment...",The companies have written to the National Pay...,ReutersPayments companies and banks are at log...,0,"Banks holding on to subsidy share, say payment..."
1,2022-04-19,Digitally ready Bank of Baroda aims to click o...,"At present, 50% of the bank's retail loans are...",AgenciesThe bank presently has 20 million acti...,1,Digitally ready Bank of Baroda aims to click o...
2,2022-05-27,Karnataka attracted investment commitment of R...,Karnataka is at the forefront in attracting in...,PTIKarnataka Chief Minister Basavaraj Bommai.K...,1,Karnataka attracted investment commitment of R...
3,2022-04-06,Splitting of provident fund accounts may be de...,The EPFO is likely to split accounts only at t...,Getty ImagesThe budget for FY22 had imposed in...,0,Splitting of provident fund accounts may be de...
4,2022-06-14,Irdai weighs proposal to privatise Insurance I...,"Set up in 2009 as an advisory body, IIB collec...",AgenciesThere is a view in the insurance indus...,1,Irdai weighs proposal to privatise Insurance I...
...,...,...,...,...,...,...
395,2022-06-10,"Banks take a cue from RBI, hike lending rates",These banks raised their respective external b...,"PTIICICI Bank, Bank of Baroda, Punjab National...",0,"Banks take a cue from RBI, hike lending rates ..."
396,2022-06-29,Sebi issues Rs 27 lakh recovery notice to indi...,"In the event of non-payment, it will recover t...",ReutersThe logo of the Securities and Exchange...,0,Sebi issues Rs 27 lakh recovery notice to indi...
397,2022-06-06,Apollo Hospital shares drop 0.68% as Sensex ...,"A total of 10,105 shares changed hands on the ...",Getty ImagesShrikant Chouhan of Kotak Securiti...,0,Apollo Hospital shares drop 0.68% as Sensex ...
398,2022-05-16,SBI at Rs 710? What makes analysts see up to 5...,Calling the stock 'attractively valued' analys...,AgenciesThe PSU bank reported a 41.27 per cent...,1,SBI at Rs 710? What makes analysts see up to 5...


## Create a dataset dictionary

In [28]:
# Create a dataset dictionary
dataset = Dataset.from_pandas(news[["full_text", 'label']])

# Display the dataset dictionary    
dataset

Dataset({
    features: ['full_text', 'label'],
    num_rows: 400
})

## Split the dataset into training and testing sets

In [29]:
# We will split the dataset into 80% training and 20% testing
train_test = dataset.train_test_split(test_size=0.2, seed=42)

In [30]:
# Now we will split the test set into 50% validation and 50% test
valid_test = train_test['test'].train_test_split(test_size=0.5, seed=42)

## Create a final dataset dictionary

In [31]:
# Combine the training, validation, and testing sets into a single dictionary
final_dataset = DatasetDict({
    'train': train_test['train'],
    'validation': valid_test['train'],
    'test': valid_test['test']
})

final_dataset

DatasetDict({
    train: Dataset({
        features: ['full_text', 'label'],
        num_rows: 320
    })
    validation: Dataset({
        features: ['full_text', 'label'],
        num_rows: 40
    })
    test: Dataset({
        features: ['full_text', 'label'],
        num_rows: 40
    })
})

## Defining models

In [32]:
# Models names
model_names = [
    "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis",
    "distilbert-base-uncased",
    "ProsusAI/finbert"
]

# Hugging Face organization name for the models saving
huggingface_owner = "rnribeiro/"

# Create a models dictionary
models = {
    model: {
        'ft_saving_dir': huggingface_owner + "FT-" + model.replace("/", "-"),
        'tokenizer': AutoTokenizer.from_pretrained(model),
        'ft_dataset': DatasetDict({
            'train': train_test['train'],
            'validation': valid_test['train'],
            'test': valid_test['test']
        }),  # Dataset for Fine-tuning
    }
    for model in model_names
}


## Tokenization of the dataset

In [33]:
# Function to tokenize the dataset
def ft_tokenize_function(model, examples):
    return model['tokenizer'](examples['full_text'], truncation=True, padding='max_length')

# Tokenize the dataset
for model in model_names:
    models[model]['ft_dataset'] = models[model]['ft_dataset'].map(
        lambda examples: ft_tokenize_function(models[model], examples),
        batched=True)

models[model_names[0]]['ft_dataset']

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['full_text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 320
    })
    validation: Dataset({
        features: ['full_text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 40
    })
    test: Dataset({
        features: ['full_text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 40
    })
})

## Loading models

This model is a distilled version of the RoBERTa-base model. It follows the same training procedure as DistilBERT.
The model has 6 layers, 768 dimension and 12 heads, totalizing 82M parameters (compared to 125M parameters for RoBERTa-base). On average DistilRoBERTa is twice as fast as Roberta-base.

In [34]:
# Load the metric
metric = evaluate.load("accuracy")


# Define the function to compute the metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


# Load all models
for model in model_names:
    # Load the model for fine-tuning
    models[model]['ft_model'] = AutoModelForSequenceClassification.from_pretrained(
        model, num_labels=2, ignore_mismatched_sizes=True
    )

    # Define the training arguments
    models[model]['ft_training_args'] = TrainingArguments(
        output_dir="./ft_training_results/" + model.replace("/", "-"),
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        push_to_hub=True,
        hub_model_id=models[model]['ft_saving_dir'],
    )

    # Define the data collator
    models[model]['ft_data_collator'] = DataCollatorWithPadding(tokenizer=models[model]['tokenizer'])

    # Define the trainer
    models[model]['ft_trainer'] = Trainer(
        model=models[model]['ft_model'],
        args=models[model]['ft_training_args'],
        train_dataset=models[model]['ft_dataset']['train'],
        eval_dataset=models[model]['ft_dataset']['validation'],
        data_collator=models[model]['ft_data_collator'],
        compute_metrics=compute_metrics,
        tokenizer=models[model]['tokenizer']
    )

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\rnrib\miniconda3\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batc

## Fine-tuning the models

In [ ]:
# Train the model
ft_train = True
if ft_train:
    for model in model_names:
        print(f"Training {model}...")
        models[model]['ft_trainer'].train()

Training mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis...


Epoch,Training Loss,Validation Loss


In [ ]:
# Evaluate the model
if ft_train:
    for model in model_names:
        print(f"Evaluating {model}...")
        models[model]['ft_trainer'].evaluate()

In [ ]:
# Save the model to the hub
if ft_train:
    for model in model_names:
        print(f"Pushing {model} to Hugging Face...")
        models[model]['ft_trainer'].push_to_hub()

## Loading trained models

In [ ]:
for model in model_names:
    # Load the tokenizer for the fine-tuned model
    models[model]['tokenizer_loaded'] = AutoTokenizer.from_pretrained(models[model]['ft_saving_dir'])

    # Load the model
    models[model]['ft_model_loaded'] = AutoModelForSequenceClassification.from_pretrained(models[model]['ft_saving_dir'],
                                                                                    num_labels=2)
    # Define the trainer
    models[model]['ft_trainer_loaded'] = Trainer(
        model=models[model]['ft_model_loaded'],
        tokenizer=models[model]['tokenizer_loaded'],
        compute_metrics=compute_metrics,
    )

## Evaluate the model

In [ ]:
# Predict the test set
for model in model_names:
    print(f"Predicting {model}...")
    models[model]['ft_predictions'] = models[model]['ft_trainer_loaded'].predict(models[model]['ft_dataset']['test'])

In [ ]:
# Display the predictions
models[model_names[0]]['ft_predictions']

## Compute relevant metrics

In [ ]:
# Initialize a dictionary to store the results
results_ft = {}

# Iterate over the model names
for model in model_names:
    # Get the predicted labels
    predicted_labels = np.argmax(models[model]['ft_predictions'].predictions, axis=-1)

    # Get the true labels
    true_labels = models[model]['ft_predictions'].label_ids

    # Compute the confusion matrix
    confusion = confusion_matrix(true_labels, predicted_labels)

    # Compute the accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)

    # Compute the precision
    precision = precision_score(true_labels, predicted_labels)

    # Compute the recall
    recall = recall_score(true_labels, predicted_labels)

    # Compute the F1 score
    f1 = f1_score(true_labels, predicted_labels)

    # Store the results in the dictionary
    results_ft[model] = {
        'confusion_matrix': confusion,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

In [ ]:
# Save the results to a dataframe
results_transformers = []
for model, metrics in results_ft.items():
    results_transformers.append({
        'Model': model,
        'Feature Representation': 'N/A',
        'Columns': 'text',
        'F1-score': metrics['f1_score'],
        'Accuracy': metrics['accuracy'],
        'Precision': metrics['precision'],
        'Recall': metrics['recall'],
    })

# Create a dataframe from the results
results_transformers_ft = pd.DataFrame(results_transformers)

results_transformers_ft.sort_values(by='F1-score', ascending=False)

## Plot the confusion matrix for the best model

In [ ]:
# Plot the confusion matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Define the labels
labels = ['Negative', 'Positive']

# Get the name of the model with the highest F1-score in results_transformers_ft
best_model = results_transformers_ft.loc[results_transformers_ft['F1-score'].idxmax()]['Model']

# Get the confusion matrix for the best model
confusion_matrix = results_ft[best_model]['confusion_matrix']

# Create the confusion matrix plot
plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix, annot=True, fmt='d', xticklabels=labels, yticklabels=labels, cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

## Load results from previous assignment

In [ ]:
# Load the results from the previous assignment to a list of dictionaries
previous_results = pd.read_csv('results/assignment1_comparison.csv')

# Display the previous results
previous_results

In [ ]:
# Concatenate the results
results = pd.concat([previous_results.copy(), results_transformers_ft], ignore_index=True)

# Save the results to a CSV file
results.to_csv('results/assignment2_comparison.csv', index=False)

# Sort the results by f1-score
results = results.sort_values(by='F1-score', ascending=False)

results

## Fine-tuning after domain adaptation

### Tokenization of the dataset

In [ ]:
# Tokenization function for domain adaptation dataset
def da_tokenize_function(model, examples):  
    return model['da_ft_tokenizer'](examples["full_text"], truncation=True, padding='max_length')

for model in model_names:
    # Define the saving directory of the domain adaptation model
    models[model]['da_saving_dir'] = huggingface_owner + "DA-" + model.replace("/", "-")
    
    # Load the tokenizer
    models[model]['da_ft_tokenizer'] = AutoTokenizer.from_pretrained(models[model]['da_saving_dir'])
    
    # Create a dataset dictionary for fine-tuning after domain adaptation
    models[model]['da_ft_dataset'] = DatasetDict({
        'train': train_test['train'],
        'validation': valid_test['train'],
        'test': valid_test['test']
    })
    
    # Tokenize the dataset
    models[model]['da_ft_dataset'] = models[model]['da_ft_dataset'].map(
        lambda examples: da_tokenize_function(models[model], examples),
        batched=True
    )

### Loading models for fine-tuning after domain adaptation

In [ ]:
for model in model_names:
    # Define the saving directory
    models[model]['da_ft_saving_dir'] = huggingface_owner + "DA-FT-" + model.replace("/", "-")

    # Load the model
    models[model]['da_ft_model'] = AutoModelForSequenceClassification.from_pretrained(models[model]['da_saving_dir'],
                                                                                      num_labels=2)

    # Define the training arguments
    models[model]['da_ft_training_args'] = TrainingArguments(
        output_dir="./da_ft_training_results/" + model.replace("/", "-"),
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        push_to_hub=True,
        hub_model_id=models[model]['da_ft_saving_dir'],
    )

    # Define the data collator
    models[model]['da_ft_data_collator'] = DataCollatorWithPadding(tokenizer=models[model]['da_ft_tokenizer'])

    # Define the trainer
    models[model]['da_ft_trainer'] = Trainer(
        model=models[model]['da_ft_model'],
        args=models[model]['da_ft_training_args'],
        train_dataset=models[model]['da_ft_dataset']['train'],
        eval_dataset=models[model]['da_ft_dataset']['validation'],
        data_collator=models[model]['da_ft_data_collator'],
        compute_metrics=compute_metrics,
        tokenizer=models[model]['da_ft_tokenizer']
    )



### Fine-tuning the models after domain adaptation

In [ ]:
da_ft_train = True
if da_ft_train:
    # Train the models
    for model in model_names:
        print(f"Training {model}...")
        models[model]['da_ft_trainer'].train()

### Evaluating the models after fine-tuning

In [ ]:
if da_ft_train:
    # Evaluate the models
    for model in model_names:
        print(f"Evaluating {model}...")
        models[model]['da_ft_trainer'].evaluate()

### Saving the models to the hub after fine-tuning

In [ ]:
if da_ft_train:
    # Save the models to the hub
    for model in model_names:
        print(f"Pushing {model} to Hugging Face...")
        models[model]['da_ft_trainer'].push_to_hub()

### Loading trained models after fine-tuning

In [ ]:
for model in model_names:
    # Load the tokenizer
    models[model]['da_ft_tokenizer_loaded'] = AutoTokenizer.from_pretrained(models[model]['da_ft_saving_dir'])

    # Load the model
    models[model]['da_ft_model_loaded'] = AutoModelForSequenceClassification.from_pretrained(
        models[model]['da_ft_saving_dir'], num_labels=2)
    # Define the trainer
    models[model]['da_ft_trainer_loaded'] = Trainer(
        model=models[model]['da_ft_model_loaded'],
        tokenizer=models[model]['da_ft_tokenizer_loaded'],
        compute_metrics=compute_metrics,
    )

### Predicting the test set

In [ ]:
for model in model_names:
    # Predict the test set
    print(f"Predicting {model}...")
    models[model]['da_predictions'] = models[model]['da_ft_trainer_loaded'].predict(models[model]['da_ft_dataset']['test'])

In [ ]:
# Display the predictions
models[model_names[0]]['da_predictions']

### Compute relevant metrics


In [ ]:
# Initialize a dictionary to store the results
results_da = {}

# Iterate over the model names
for model in model_names:
    # Get the predicted labels
    predicted_labels = np.argmax(models[model]['da_predictions'].predictions, axis=-1)

    # Get the true labels
    true_labels = models[model]['da_predictions'].label_ids

    # Compute the confusion matrix
    confusion = confusion_matrix(true_labels, predicted_labels)

    # Compute the accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)

    # Compute the precision
    precision = precision_score(true_labels, predicted_labels)

    # Compute the recall
    recall = recall_score(true_labels, predicted_labels)

    # Compute the F1 score
    f1 = f1_score(true_labels, predicted_labels)

    # Store the results in the dictionary
    results_da[model] = {
        'confusion_matrix': confusion,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

### Save the results to a dataframe

In [ ]:
results_transformers_da = []
for model, metrics in results_da.items():
    results_transformers_da.append({
        'Model': "DA-" + model,
        'Feature Representation': 'N/A',
        'Columns': 'text',
        'F1-score': metrics['f1_score'],
        'Accuracy': metrics['accuracy'],
        'Precision': metrics['precision'],
        'Recall': metrics['recall'],
    })

# Create a dataframe from the results
results_da = pd.DataFrame(results_transformers_da)

results_da

### Compare with previous results

In [ ]:
# Concatenate the results
results = pd.concat([results, results_da], ignore_index=True)

# Save the results to a CSV file
results.to_csv('results/assignment2_comparison_da.csv', index=False)

# Sort the results by f1-score
results = results.sort_values(by='F1-score', ascending=False)

results